In [1]:
import redis
import time
import threading

In [2]:
# Connect to Redis
redis = redis.Redis(host='localhost', port=6379, db=0)

## Configuration 1

We setup a Redis key with an expiration of 5 seconds, and then we check immediately after if they key exists, which we expect it to do.

Then we sleep the thread for 6 seconds so our key should have expired, and we check again if the key exists, which we expect it to not exist.

In [3]:
# Set a key with 5-second expiration
redis.set('temp_key', 'this will expire', ex=5)

# Check right after SET
print("Before expiration:", redis.get('temp_key').decode())

# Wait past expiration
time.sleep(6)

# Check again after expiration
print("After expiration:", redis.get('temp_key'))  # Should be None

Before expiration: this will expire
After expiration: None


## Configuration 5

We have made one Subscriber that will listen to new game updates, and then we have 2 Publishers who will start up a new game, and then we expect the Subscriber to have received the notifications from our two Publishers.

In [4]:
def subscriber():
    pubsub = redis.pubsub()
    pubsub.subscribe('game_updates')
    for message in pubsub.listen():
        if message['type'] == 'message':
            print("Received:", message['data'].decode())

def publisher():
    time.sleep(1) # Ensure subscriber is ready
    redis.publish('game_updates', 'User is now playing Counter-Strike 2')

def publisher_2():
    time.sleep(1.5) # Ensure subscriber is ready
    redis.publish('game_updates', 'User2 is now playing Counter-Strike 2')

# Start threads
threading.Thread(target=subscriber).start()
threading.Thread(target=publisher).start()
threading.Thread(target=publisher_2).start()

# Wait to see output
time.sleep(2)

Received: User is now playing Counter-Strike 2
Received: User2 is now playing Counter-Strike 2
